# Introducción al análisis predictivo con Regresión

## Objetivos

1. Conocer los conceptos de Correlación, causalidad y dependencia y función lineal.
2. Entender el modelo de regresión lineal y sus condiciones a cumplir. 
3. Conocer cómo usarlo.


## Aproximación Clásica

In [ ]:
import numpy as np
import statsmodels.api as sm

spector_data = sm.datasets.spector.load()
spector_data.exog = sm.add_constant(spector_data.exog, prepend=False)
print(spector_data.endog)

In [ ]:
import numpy as np
import statsmodels.api as sm

spector_data = sm.datasets.spector.load()

spector_data.exog = sm.add_constant(spector_data.exog, prepend=False)

# Ajustar un modelo de mínimos cuadrados generalizados (Ordinary Least Squares -- OLS)
mod = sm.OLS(spector_data.endog, spector_data.exog)

res = mod.fit()

print(res.summary())

## 1. Modelos de *machine learning*

El propósito de los modelos de *machine learning* es encontrar características en datos conocidos (del pasado), ya sea para comprender algún fenómeno o para hacer alguna estimación sobre datos nuevos (del futuro). En cualquier caso, estos modelos son "entrenados" con base en datos observados.

Los modelos de *machine learning* comúnmente se clasifican en dos categorias: supervisados, los cuales cuentan con una variable de respuesta que se quiere predecir; no supervisados, los cuales se enfocan en encontrar relaciones subyacentes entre los datos sin necesidad de una variable de respuesta. Por ejemplo, si tenemos imágenes médicas acompañadas del diagnóstico de un profesional, quien indica si existe una patología o no, podriamos plantear un modelo de aprendizaje supervisado. Si clasificamos las observaciones en estas dos categorías (patología, no patología), podemos utilizarlas para "entrenar" nuestro modelo y posteriormente determinar, sin la ayuda del médico, si nuevas imágenes corresponden a la patología o no.

El aprendizaje supervisado nos permite abordar dos tipos de problemas: regresión y clasificación. En los problemas de regresión el objetivo es predecir una variable de respuesta que toma valores continuos (p.ej., estimar el precio de una casa con base en sus características), mientras que en los problemas de clasificación el objetivo es predecir una variable de respuesta que toma valores discretos (p.ej., estimar si una imagen corresponde a un objeto u otro).

El aprendizaje no supervisado nos permite abordar dos tipos de problemas: clustering y reducción de dimensionalidad. En los problemas de clustering el objetivo es encontrar conjuntos de observaciones con características similares que podrían pertenecer a una misma clase o grupo. En los problemas de reducción de dimensionalidad el objetivo es encontrar las características que representan la información relevante de un conjunto de datos de la manera más concisa.

### 1.1. Pasos típicos para desarrollar un modelo en el paquete scikit-learn

Scikit-learn es un paquete que provee implementaciones eficientes de muchos algoritmos comunes de *machine learning* con una API (interfaz de programación de aplicaciones) sencilla de utilizar. En general, llevamos a cabo los siguientes pasos al momento de crear un modelo:

**1. Importar un módulo del paquete `sklearn` que contiene el modelo que vamos a utilizar:** por ejemplo, si queremos desarrollar un modelo de regresión lineal, debemos importar el módulo correspondiente: `linear_model`.
~~~ python
from sklearn import linear_model
~~~

**2. Elegir los datos que queremos analizar:** en general, podemos pensar en los datos de entrada de un modelo de *machine learning* como una matriz, que llamaremos $X$, con $m$ filas y $n$ columnas, donde $m$ corresponde al número de muestras u observaciones (*samples*) y $n$ corresponde al número de variables o características (*features*). En el caso de modelos no supervisados, esa matriz será suficiente. Para un modelo supervisado, requerimos una columna adicional de datos, $y$ (con $m$ filas, una por muestra), correspondientes a la variable de respuesta. Supongamos que contamos con un `DataFrame` (`DF`) en el que las columnas 0 a 4 son variables independientes (*features*) y la columna 5 es la variable de respuesta. La selección de datos para la regresión será:
~~~python
X = DF.iloc[:,:5]
y = DF.iloc[:,5]
~~~

**3. Crear conjuntos de datos para entrenamiento y prueba:** luego de dividir los datos en $X$ y en $y$, debemos crear un conjunto de datos de entrenamiento, con el cual se entrenará el modelo, y un conjunto de datos de prueba, con el cual se verificará la efectividad del modelo. Es fundamental llevar a cabo este paso ya que, de lo contrario, estaríamos entrenando y probando al modelo con el mismo set de datos, lo cual no nos permitiría saber si el modelo funciona correctamente o si simplemente se “aprendió” los resultados óptimos. Una analogía útil al momento de pensar en esta situación es la de un profesor que le muestra las respuestas de un parcial a sus estudiantes y luego los evalúa con las mismas preguntas. Normalmente utilizamos el 80% de los datos para entrenar el modelo y el 20% para probarlo.
~~~python
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
~~~

**4. Declarar una instancia del modelo:** cada módulo dispone de diversos modelos. Por ejemplo, dentro de `linear_model` podemos encontrar la clase `LinearRegression`, que contiene los elementos necesarios para desarrollar un modelo de regresión lineal. Para proceder, debemos declarar un objeto de la clase `LinearRegression` como se muestra a continuación.
~~~python
mi_modelo = linear_model.LinearRegression()
~~~

**5. Entrenar el modelo:** el objeto `mi_modelo` cuenta con los métodos y atributos necesarios para desarrollar un modelo de regresión. El método más común en los modelos de *machine learning* es `fit`, pues es el encargado de entrenar el modelo de acuerdo a los datos suministrados para encontrar los parámetros que mejor se ajustan. A contiuación vemos la instrucción para entrenar el modelo:
~~~python
mi_modelo.fit(X_train,y_train)
~~~

**6. Consultar los resultados del modelo:** en el caso del modelo de regresión lineal, nos interesa conocer los coeficientes que acompañan a cada una de las variables del modelo. Cuando ejecutamos el método `fit`, estos valores son almacenados en el atributo `coef_`; el caracter ( _ ) es una convención para los atributos calculados internamente por el método `fit`. Así, podemos consultar estos valores de la siguiente manera:
~~~python
print(mi_modelo.coef_)
~~~

**7. Utilizar el modelo para predecir:** por lo general, queremos aplicar nuestro modelo entrenado a nuevas observaciones de datos. Para esto utilizamos el método `predict`, que recibe por parámetro las columnas correspondientes a variables independientes ($X$) y genera las predicciones correspondientes para la variable de respuesta ($y$). A continuación, utilizamos los datos de prueba (`X_test`) para predecir la variable de respuesta.
~~~python
y_estimado = mi_modelo.predict(X_test)
~~~

**8. Evaluar el desempeño del modelo:** es importante asegurarnos que el modelo que hemos entrenado se desempeña bien con datos nuevos y no se ha sobreajustado con los datos de entrenamiento. En este caso, lo usual es medir qué tan cercanas son las predicciones hechas por el modelo (`y_estimado`) a los valores reales de la variable de respuesta de los datos de prueba (`y_test`). En el caso de la regresión lineal, una métrica común es el $R^2$, el cual puede ser calculado con el método `r2_score` del módulo `sklearn.metrics`. Procedemos a importar la funcion y estimar la métrica:
~~~python
from sklearn.metrics import r2_score
mi_r2 = r2_score(y_test, y_estimado)
~~~

También podemos usar el método `score` para conocer la precisión del modelo a partir de los datos de prueba:
~~~python
mi_r2 = mi_modelo.score(X_test, y_test)
~~~  

## 2. Modelos de regresión
Un modelo de regresión es un modelo matemático que busca determinar la relación de una variable dependiente $Y$ con respecto a una o más variables independientes $X$. Para ello debemos postular una ecuación que relacione estas variables. Por ejemplo, queremos saber si el rendimiento de un producto químico está relacionado con su temperatura. Podemos construir un modelo de regresión para identificar la relación que existe entre las variables y predecir el rendimiento de un producto dada una temperatura.

Veremos tres tipos de modelos de regresión:
1.	Regresión lineal simple
2.	Regresión múltiple
3.	Regresión logística

En este tutorial veremos cómo implementar estos modelos en el contexto de analítica de datos sin explicar su trasfondo estadístico, veremos esta explicación en otros cursos de la Maestría.

### 2.1. Regresión lineal simple

Una regresión lineal simple se basa en una sola variable independiente $X$, una variable dependiente $y$ y supone que existe una relación lineal entre estas. La ecuación general para este tipo de regresión es la siguiente:$$y = β_0 + β_1X + e$$

+ $β_0$ es la altura a la que la recta corta al eje Y, la cual se denomina término independiente.

+ $β_1$ es el incremento que se produce en la variable $y$ cuando la variable $X$ aumenta una unidad, el cual se denomina pendiente de la recta.

<a id='ej1'></a>
##### Ejemplo 1
En este ejemplo utilizaremos los datos de vivienda en Boston y seguiremos los pasos vistos anteriormente, para llevar a cabo una regresión lineal que nos permita identificar cuál es la relación entre el precio de venta de una casa y el número de habitaciones que tiene.

Empecemos cargando los datos y filtrando la información relevante:

In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt

# Importamos los datos:
data_boston = "http://lib.stat.cmu.edu/datasets/boston"

# Creamos un DataFrame para visualizarlos:
df = pd.read_csv(data_boston, sep="\s+", skiprows=22, header=None)
df

In [ ]:
data = np.hstack([df.values[::2, :], df.values[1::2, :2]])
target = df.values[1::2, 2]

In [ ]:
# Elegimos los datos que queremos analizar

# Definimos la variable independiente como el número de habitaciones (columna 5 de los datos)
X = data[:, np.newaxis, 5]

# Definimos la variable de respuesta utilizando el atributo target del dataset de sklearn
y = target

Ahora, visualicemos los datos para darnos una idea de si existe una relación lineal entre las variables.

In [ ]:
# Graficamos los datos correspondientes:
plt.scatter(X, y)
plt.xlabel('Número de habitaciones')
plt.ylabel('Precio')
plt.show()

Después, creamos el modelo de regresión siguiendo los pasos vistos anteriormente:

In [ ]:
from sklearn.model_selection import train_test_split

# Creamos conjuntos de datos para entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Declaramos una instancia del modelo:
lr = linear_model.LinearRegression()

# Entrenamos el modelo:
lr.fit(X_train, y_train)

# Utilizamos el modelo para predecir
y_estimado = lr.predict(X_test)

# Graficamos los datos y las predicciones hechas por el modelo
plt.scatter(X_test, y_test)
plt.plot(X_test, y_estimado, color='red', linewidth=3)
plt.xlabel('Número de habitaciones')
plt.ylabel('Precio')
plt.show()

Como podemos ver, parece que el modelo de regresión se ajusta bien a los datos de vivienda de Boston. Para comprobarlo, evaluemos el desempeño del modelo:

In [ ]:
lr.score(X_test, y_test)

El desempeño del modelo no es muy alto. Esto nos indica que, aunque a simple vista pareciera que el número de habitaciones es suficiente para estimar el precio de una vivienda, es probable que necesitemos considerar más variables con el fin de encontrar una mejor aproximación del precio de una casa.

### 2.2. Regresión lineal múltiple

Una regresión lineal múltiple es el caso de una regresión lineal simple con múltiples variables independientes $X$. La ecuación general para este tipo de regresión es la siguiente:$$y = β_0 + β_1X_1 + β_2X_2 + ... + β_nX_n + e$$

El objetivo de este tipo de regresión es incorporar la información de varias variables para explicar conjuntamente el comportamiento de la variable de respuesta.

##### Ejemplo 2.
Retomemos el [Ejemplo 1](#ej1). Esta vez queremos predecir el precio de una casa a partir de múltiples variables. Para esto usaremos todas las variables que existen en el set de datos de vivienda de Boston. Entre estas variables encontramos el número de habitaciones de la vivienda, número de crímenes en el área cercana, costo de impuestos, y algunas más.

De este modo, empezaremos cargando el set de datos y separando la variable de interes de las otras variables:

In [1]:
# import numpy as np
# from sklearn import linear_model
# import matplotlib.pyplot as plt

# # Importamos los datos:
# data_boston = datasets.load_boston()

# # Elegimos los datos que queremos analizar

# # Definimos todas las variables como variables independientes
# X = data_boston.data

# # Definimos la variable de respuesta utilizando el atributo target del dataset de sklearn
# y = data_boston.target

import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt

data_url = "http://lib.stat.cmu.edu/datasets/boston"
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None, names=column_names)

raw_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.00,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,NaN,NaN,NaN
1,396.90000,4.98,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.02731,0.00,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,NaN,NaN,NaN
3,396.90000,9.14,21.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.02729,0.00,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,396.90000,5.64,23.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008,0.10959,0.00,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,NaN,NaN,NaN
1009,393.45000,6.48,22.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,0.04741,0.00,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,NaN,NaN,NaN


In [7]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
data = pd.read_csv(url)
data

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


Luego, debemos crear el modelo de la misma manera que en la regresión lineal simple.

In [ ]:
from sklearn.model_selection import train_test_split

# Creamos conjuntos de datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Declaramos una instancia del modelo
lr = linear_model.LinearRegression()

# Entrenamos el modelo
lr.fit(X_train, y_train)

# Utilizamos el modelo para predecir
y_pred = lr.predict(X_test)

Finalmente, evaluemos el desempeño del modelo:

In [ ]:
lr.score(X_test, y_test)

Como podemos observar, el modelo obtuvo una precisión mucho mayor que cuando solo se usó el número de habitaciones. A partir de este resultado, podemos concluir que el precio de una casa en Boston puede ser explicado conjuntamente por numerosos factores y no por uno en particular.

In [ ]:
spector_data.endog